In [ ]:
%matplotlib inline

import suPAErnova as snpae
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Some useful paths to have access to
cwd = Path.cwd()
examples_dir = cwd.parent.parent.parent.parent
data_dir = examples_dir / "suPAErnova_data" # Put your data into this directory

# Global Configuration

In [ ]:
verbose = False # Increase log verbosity
force = False # Don't rerun steps if not necessary

cfg = snpae.setup_global_config({}, verbose=verbose, force=force) # Just pass an empty dictionary to initialise

# Order of Operations
To run any `PAE` step, you must first run the `DATA` step, storing its results for later use. If you try to run a `PAE` step anyway, you'll get the following error.

In [ ]:
cfg["TF_PAE"] = {}
_tf_pae = snpae.steps.TF_PAEStep(cfg)

# Data Configuration and Setup

In [ ]:
# Note that these keys *MUST* be captilalised. This is not the case when using a `suPAErnova.toml` config file.
data_cfg = {
    # === Required Keys ===

    # Path to directory containing data.
    #   Can be absolute or relative to the base path.
    "DATA_DIR": str(data_dir), # Needs to be a string so that SuPAErnova can validate it
    
    # Metadata CSV containing SN names and SALT fit parameters.
    #   Can be absolute or relative to the data path.
    "META": "meta.csv",

    # TXT file containing additional SALT fit parameters.
    #   Can be absolute or relative to the data path.
    "IDR": "IDR_eTmax.txt",

    # TXT file containing a mask of bad spectra / wavelength ranges.
    #   Can be absolute or relative to the data path.
    "MASK": "mask_info_wmin_wmax.txt",
}

cfg["DATA"] = data_cfg
data = snpae.steps.DATAStep(cfg)
success, result = data.setup()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error running setup: {result}")
success, result = data.run()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error running: {result}")
success, result = data.result()
if not success: # Make sure you handle failures appropriately!
    data.log.error(f"Error saving results: {result}")

# The result() function of a step returns the original cfg passed in, but now with the step stored in cfg["GLOBAL"]["RESULTS"]step.name]
# This allows later steps to access the results of previous steps.
cfg = result
print(cfg["GLOBAL"]["RESULTS"][data.name])

# TF_PAE Configuration

In [ ]:
# Note that these keys *MUST* be captilalised. This is not the case when using a `suPAErnova.toml` config file.
tf_pae_cfg = { # TODO: Reorder / Reformat / Rename these keys to make it easy to handle
    # Path to a colourlaw file. If provided, will be used as the kernel initialiser in the decoder.
    #   Defaults to "" (i.e. no colourlaw file)
    "colourlaw": "",

    # Which kfold (or kfolds if passing a list) to use
    #   Defaults to 0
    "kfold": 0,

    # What fraction of training data to split into validation data. Set to 0.0 to use test data as validation data.
    #   Defaults to 0.0
    "validation_frac": 0.0,

    # Number of epochs to use when training ΔAᵥ
    #   Defaults to 1000
    "epochs_colour": 1000,

    # Number of epochs to use when training ΔAᵥ and any number of non-physical latent parameters
    #   Defaults to 1000
    "epochs_latent": 1000,

    # Number of epoch to use when training ΔAᵥ all non-physical latent parameters, and Δℳ
    #   Defaults to 1000
    "epochs_amplitude": 1000,

    # Number of epoch to use when training ΔAᵥ all non-physical latent parameters, Δℳ, and Δ𝓅 (i.e. all parameters)
    #   Defaults to 5000
    "epochs_all": 5000,

    # Split training into several stages (true), or train all parameters at once (false).
    #   When splitting, the result of the previous stage will be used as the starting point for the next stage.
    #   When splitting, the following stages are run:
    #   If physical_latent:
    #       Stage 0: Train ΔAᵥ
    #   If latent_dim > 0:
    #       Stage latent_dim - (latent_dim - 1): Train ΔAᵥ and the first non-physical latent parameter
    #       Stage latent_dim - (latent_dim - 2): Train ΔAᵥ and the first two non-physical latent parameters
    #       ...
    #       Stage latent_dim: Train ΔAᵥ and all non-physical latent parameters
    #   If physical_latent:
    #       Stage latent_dim + 1: Train ΔAᵥ, all non-physical latent parameters, and Δℳ
    #       Stage latent_dim + 2: Train ΔAᵥ, all non-physical latent parameters, Δℳ, and Δ𝓅
    #   Defaults to true
    "split_training": True,

    # Minimum redshift for spectral data.
    #   Defaults to 0.02
    "min_train_redshift": 0.02,

    # Maximum redshift for spectral data.
    #   Defaults to 1.0
    "max_train_redshift": 1.0,

    # Minimum phase for spectral data, relative to peak. Spectral data earlier than this phase will be cut. Defaults to -10
    "min_train_phase": -10,

    # Maximum phase for spectral data, relative to peak. Spectral data later than this phase will be cut.
    #   Defaults to 40
    "max_train_phase": 40,

    # Loss function to use during training. Possible values include:
    #   - "MAE": [Mean Absolute Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanAbsoluteError)
    #   - "WMAE": Weighted [Mean Absolute Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanAbsoluteError)
    #   - "MSE": [Mean Squared Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
    #   - "WMSE": Weighted [Mean Squared Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
    #   - "RMSE": Root [Mean Squared Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
    #   - "WRMSE": Weighted Root [Mean Squared Error](https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError)
    #   - "NGLL": Negative Gaussian Log Likelihood
    #   - "HUBER": [Huber](https://www.tensorflow.org/api_docs/python/tf/keras/losses/Huber)
    #   - "WHUBER": Weighted [Huber](https://www.tensorflow.org/api_docs/python/tf/keras/losses/Huber)
    #   - "MAGNITUDE": Custom loss function comparing spectral magnitudes
    #   - "NULL": Custom loss function which returns None no matter what. Designed for use with custom user-defined loss functions, integrated through callbacks.
    #   Defaults to WMAE
    "loss": "WMAE",

    # Scaling factor which punishes the overall difference between real spectra and their corresponding encode-decode spectra.
    #   Defaults to 500.0
    "loss_amplitude_offset": 500.0,

    # Scaling factor which encourages median Δ𝓅 close to 1
    #   Defaults to 1000.0
    "loss_amplitude_parameter": 1000.0,

    # Scaling factor applied to the covariant loss between physical model parameters
    #   Defaults to 50000.0
    "loss_covariance": 50000.0,

    # Ignore correlations with dust (i.e. colour)
    #   TODO: Better explanation
    #   Defaults to true
    "decorrelate_dust": True,

    # Ignore correlations
    #   TODO: Better explanation
    #   Defaults to true
    "decorrelate_all": True,

    # Set the size of each batch the data will be randomly split between.
    #   Must evenly split the data.
    #   TODO: Replace batch_size with num_batches which will then split the data as evenly as possible between that many batches. Should result in a better user experience
    #   Defaults to 57, the size of the batches used in the original SuPAErnova examples
    "batch_size": 57,

    # Controls the scale of random offsets applied to the training data's amplitude parameter.
    #   Will apply a gaussian offset with width determined by the observational uncertainty, scaled by `noise_scale`.
    #   Thus a noise_scale of 0.0 will not apply any offset.
    #   Defaults to 1.0
    "noise_scale": 1.0,

    # The fraction of spectra to randomly mask throughout training.
    #   Defaults to 0.1
    "mask_frac": 0.1,

    # Controls the scale of random offsets applied to the training data's phase parameter.
    #   If sigma_time is a positive value (`n`), apply a gaussian offset with constant width `n / 50`.
    #   If sigma_time is 0, apply a gaussian offset with width determined by SALT constraints (`dphase / 50`).
    #   If sigma_time is -1, no offset will be applied.
    #   Defaults to 0.3
    "sigma_time": 0.3,

    # Whether to save intermediate models during training.
    #   Defaults to true
    "save_model": True,

    # In between training stages, load the best model of the previous stage (if true), or the final epoch of the previous stage (if false).
    #   Defaults to false
    "load_best": False,

    # Whether to use the ["DENSE"](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense) or ["CONVOLUTIONAL"](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) model architecture.
    #   Defaults to "DENSE"
    "layer_type": "DENSE",

    # Which activation function to use. Possible values include:
    #   - "ELU": Use an [Exponential Linear Unit](https://www.tensorflow.org/api_docs/python/tf/keras/activations/elu) activation function
    #   - "GELU": Use a [Gaussian Error Linear Unit](https://www.tensorflow.org/api_docs/python/tf/keras/activations/gelu) activation function
    #   - "RELU": Use a [REctified Linear Unit](https://www.tensorflow.org/api_docs/python/tf/keras/activations/relu) activation function
    #   - "SWISH": Use a [Swish / Silu](https://www.tensorflow.org/api_docs/python/tf/keras/activations/silu) activation function
    #   - "TANH": Use a [Hyperbolic Tangent](https://www.tensorflow.org/api_docs/python/tf/keras/activations/tanh) activation function
    #   - "NULL": Custom activation function which returns None no matter what. Designed for use with custom user-defined activation functions, integrated through callbacks.
    #   Defaults to "RELU"
    "activation": "RELU",

    # What learning rate to use for training
    #   Defaults to 0.005
    "lr": 0.005,

    # What learning rate to use when training the final model, which required training all parameters simultaneously.
    #   Default to 0.001
    "lr_deltat": 0.001,
    
    # Which optimiser to use. Possible values include:
    #   - "ADAM": Use an [Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam) optimiser.
    #   - "ADAMW": Use a [Weighted Adam](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/AdamW) optimiser.
    #   - "SGD": Use a [Gradient Descent with Momentum](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD) optimiser.
    #   - "NULL": Custom optimiser which returns None no matter what. Designed for use with custom user-defined optimisers, integrated through callbacks.
    #   Defaults to "ADAMW"
    "optimiser": "ADAMW",

    # Rate of weight decay to use in your chosen optimiser
    #   Default to 0.00001
    "weight_decay_rate": 0.00001,

    # Which scheduler to use. Possible values include:
    #   - "EXPONENTIAL": Use an [Exponential Decay](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay) scheduler.
    #   - "IDENTITY": Use an Identity scheduler, which just returns the provided learning rate.
    #   - "NULL": Custom scheduler which returns None no matter what. Designed for use with custom user-defined schedulers, integrated through callbacks.
    #   Defaults to "EXPONENTIAL"
    "scheduler": "EXPONENTIAL",

    # Rate of learning rate decay to use in your chosen scheduler
    #   Defaults to 0.95
    "lr_decay_rate": 0.95,

    # Number of learning rate decay steps to use in your chosen scheduler
    #   Defaults to 300
    "lr_decay_steps": 300,

    # [L2 Regularisation](https://www.tensorflow.org/api_docs/python/tf/keras/regularizers/L2)  penalty to apply during training, or 0.0 for no penalty.
    #   Defaults to 0.0
    "kernel_regulariser": 0.0,

    # Whether to include a [Dropout Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) to the encoder, with a rate of `dropout`. Setting this to 0.0 disables the Dropout Layer.
    #   Defaults to 0.0
    "dropout": 0.0,

    # Whether to include a [Batch Normalisation Layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization) to the encoder.
    #   Defaults to false
    "batchnorm": False,

    # Whether to include physical latent parameters.
    #   Defaults to true, which corresponds to three physical latent parameters:
    #     - Δ𝓅: Different in time of peak brightness relative to the SALT fits (labelled as `phase` throughout the source code)
    #     - Δℳ : Magnitude residual (labeled as `amplitude` or `amp` throughout the source code)
    #     - ΔAᵥ: Relative extrinsic extinction (labeled as `AV` throughout the source code)
    "physical_latent": True,

    # Dimensions of each dense layer in the encoder.
    #   Default to [256, 128]
    "encode_dims": [256, 128],

    # Dimensions of each dense layer in the decoder. If an empty list is passed, the reverse of encode_dims will be used.
    #   Defaults to []
    "decode_dims": [],

    # The number of non-physical latent dimensions to encode.
    #   Defaults to 3
    "latent_dim": 3,
}

cfg["TF_PAE"] = tf_pae_cfg
tf_pae = snpae.steps.TF_PAEStep(cfg)
print(tf_pae)

# Setup

In [ ]:
success, result = tf_pae.setup()
if not success: # Make sure you handle failures appropriately!
    tf_pae.log.error(f"Error running setup: {result}")
print(tf_pae)

# Run
Warning, this will actually perform the `PAE` training and may take some time!

In [ ]:
success, result = tf_pae.run()
if not success: # Make sure you handle failures appropriately!
    tf_pae.log.error(f"Error running: {result}")
print(tf_pae)

# Results

In [ ]:
success, result = tf_pae.result()
if not success: # Make sure you handle failures appropriately!
    tf_pae.log.error(f"Error saving results: {result}")

# The result() function of a step returns the original cfg passed in, but now with the step stored in cfg["GLOBAL"]["RESULTS"]step.name]
# This allows later steps to access the results of previous steps.
cfg = result
print(cfg["GLOBAL"]["RESULTS"][tf_pae.name])

# Analysis
No automatic analyse function integrated at the moment.

In [ ]:
success, result = tf_pae.analyse()
if not success: # Make sure you handle failures appropriately!
    tf_pae.log.error(f"Error analysing: {result}")